# This is our analysis of WPI's Emissions Data

Data from Tighe & Bond Report 2020

In [1]:
#IMPORT DATA
import pandas as pd
import plotly.graph_objects as go

data22 = pd.read_csv('/Users/sydneypeno/Downloads/WPI_Utilities_Tighe.csv')
print(data22.head())

   Unit num                  Unit Name  Building Fuel/Chemical Quantity Used  \
0       4.0         Founders Boiler #4  Founders   Natural Gas     4,201,137   
1       4.0         Founders Boiler #4  Founders   Natural Gas     4,201,137   
2       4.0         Founders Boiler #4  Founders   Natural Gas     4,201,137   
3       7.0  Generator #1 Daniels Hall   Daniels   Natural Gas        36,201   
4       7.0  Generator #1 Daniels Hall   Daniels   Natural Gas        36,201   

  Unit of Mea Pollutant Emission Factor            Unit of Mea2  \
0  cubic feet       CO2         120,000  lbs CO2/106 cubic feet   
1  cubic feet       CH4          2.2708  lbs CH4/106 cubic feet   
2  cubic feet       N2O          0.2205  lbs N2O/106 cubic feet   
3  cubic feet       CO2         120,020  lbs CO2/106 cubic feet   
4  cubic feet       CH4          2.2708  lbs CH4/106 cubic feet   

  Pounds of Pollutant Emitted Tons of Pollutant Emitted  Warming Potential  \
0                     504,136         

In [2]:
#CLEAN UP DATA
# Remove commas and convert to numeric
data22['Metric Tons of CO2e Emitted'] = data22['Metric Tons of CO2e Emitted'].str.replace(',', '').astype(float)
data22['Pounds of Pollutant Emitted'] = data22['Pounds of Pollutant Emitted'].str.replace(',', '').astype(float)

df = pd.DataFrame(data22)
# Clean up the data by removing leading and trailing whitespaces from the "Pollutant" column
df['Pollutant'] = df['Pollutant'].str.strip()

We can analyze which GHG WPI emits the most of by plotting the average pounds of emissions by pollutant

In [20]:
import numpy as np

average_emissions = df.groupby('Pollutant')['Pounds of Pollutant Emitted'].mean().reset_index()

# Plot the average emissions for each pollutant with a logarithmic scale
fig = go.Figure()

fig.add_trace(go.Bar(
    x=average_emissions['Pollutant'],
    y=average_emissions['Pounds of Pollutant Emitted'],
    marker=dict(color=['lightgreen', 'darkgreen', '#B6E880']),
    text=average_emissions['Pounds of Pollutant Emitted'].apply(lambda x: f'{x:.2f}'),
    textposition='auto',
))

# Update layout
fig.update_layout(
    title='Average Pounds of Emissions per Pollutant FY 2022',
    xaxis=dict(title='Pollutant'),
    yaxis=dict(title='Average Pounds of Pollutant Emitted', type='log')
)

fig.show()

In [19]:
average_metric_emissions = df.groupby('Pollutant')['Metric Tons of CO2e Emitted'].mean().reset_index()

# Create a Plotly bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=average_metric_emissions['Pollutant'],
    y=average_metric_emissions['Metric Tons of CO2e Emitted'],
    marker=dict(color=['lightgreen', 'darkgreen', '#B6E880']),
    text=average_metric_emissions['Metric Tons of CO2e Emitted'].apply(lambda x: f'{x:.2f}'),
    textposition='auto',
))

# Update layout
fig.update_layout(
    title='Average Metric Tons of CO2e Emissions per Pollutant FY 2022',
    xaxis=dict(title='Pollutant'),
    yaxis=dict(title='Average Metric Tons of CO2e Emitted', type='log')
)

fig.show()


In the WPI data, we can observe which buildings around campus emit the most GHGs

In [18]:
import plotly.graph_objects as go

# Group by both 'Building' and 'Pollutant', then sum the 'Metric Tons of CO2e Emitted'
grouped_data = df.groupby(['Building', 'Pollutant'])['Metric Tons of CO2e Emitted'].sum().reset_index()

# Pivot the data to have 'Building' as index, 'Pollutant' as columns, and 'Metric Tons of CO2e Emitted' as values
pivot_data = grouped_data.pivot(index='Building', columns='Pollutant', values='Metric Tons of CO2e Emitted')

# Define the list of colors for the pollutants
colors = ['lightgreen', 'darkgreen', '#B6E880']

# Create traces for each pollutant
data = []
for i, pollutant in enumerate(pivot_data.columns):
    trace = go.Bar(
        x=pivot_data.index,
        y=pivot_data[pollutant],
        name=pollutant,
        marker=dict(color=colors[i]),
    )
    data.append(trace)

# Define layout
layout = go.Layout(
    title='Metric Tons of CO2e Emitted per Building FY 2022',
    xaxis=dict(title='Building'),
    yaxis=dict(title='Metric Tons of CO2e Emitted', type='log'),
    barmode='stack',
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Show plot
fig.show()


Data from Tighe & Bond Report from 2013

In [10]:
data13 = pd.read_csv('/Users/sydneypeno/Downloads/2013_Tighe.csv')
print(data13.head())

       EU#                       Emission Unit    Building     Fuel  \
0   EU # 4          Founders Equip. Boiler # 4    Founders  N. G.     
1   EU # 7  Generator # 1 Daniels Hall, Kohler     Daniels   N.G.     
2   EU # 8    Generator # 2 Founders Hall, CPI    Founders   N.G.     
3   EU # 9      Generator # 3 Harrington, Onan  Harrington   N.G.     
4  EU # 10   Generator # 4 Salisbury, Olympian   Salisbury   N.G.     

  Quantity Used Unit of Measure  Quantity Used.1 Unit of Measure.1 Pollutant  \
0    48859.0000          Therms          4885.90             MMBTU       CO2   
1        0.0263            MMCF            27.04             MMBTU       CO2   
2        0.0291            MMCF            29.91             MMBTU       CO2   
3        0.0056            MMCF             5.76             MMBTU       CO2   
4        0.5872            MMCF           603.64             MMBTU       CO2   

   Emission  Factor (EF) Unit of Measure.2       EF Source  \
0                  52.91      

In [11]:
#CLEAN UP DATA

# Clean up the data by removing leading and trailing whitespaces from the "Pollutant" column
data13['Pollutant'] = data13['Pollutant'].astype(str)
data13['Pollutant'] = data13['Pollutant'].str.strip()

df1 = pd.DataFrame(data13)

In [17]:
# Group by 'Pollutant' and calculate the mean of 'Metric-Tons of CO2 Emitted'
average_metric_emissions = df1.groupby('Pollutant')['Metric-Tons of CO2 Emitted'].mean().reset_index()

# Create a Plotly bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=average_metric_emissions['Pollutant'],
    y=average_metric_emissions['Metric-Tons of CO2 Emitted'],
    marker=dict(color=['lightgreen', 'darkgreen', '#B6E880']),
    text=average_metric_emissions['Metric-Tons of CO2 Emitted'].apply(lambda x: f'{x:.2f}'),
    textposition='auto',
))

# Update layout
fig.update_layout(
    title='Average Metric-Tons of CO2 Emitted per Pollutant FY 2013',
    xaxis=dict(title='Pollutant'),
    yaxis=dict(title='Average Metric-Tons of CO2 Emitted', type='log')
)

fig.show()


In [16]:
import plotly.graph_objects as go

# Group by both 'Building' and 'Pollutant', then sum the 'Metric-Tons of CO2 Emitted'
grouped_data = df1.groupby(['Building', 'Pollutant'])['Metric-Tons of CO2 Emitted'].sum().reset_index()

# Pivot the data to have 'Building' as index, 'Pollutant' as columns, and 'Metric-Tons of CO2 Emitted' as values
pivot_data = grouped_data.pivot(index='Building', columns='Pollutant', values='Metric-Tons of CO2 Emitted')

# Define colors for each pollutant
colors = ['lightgreen', 'darkgreen', '#B6E880']

# Create traces for each pollutant
data = []
for pollutant, color in zip(pivot_data.columns, colors):
    trace = go.Bar(
        x=pivot_data.index,
        y=pivot_data[pollutant],
        name=pollutant,
        marker=dict(color=color),
    )
    data.append(trace)

# Define layout
layout = go.Layout(
    title='Metric-Tons of CO2 Emitted per Building FY 2013',
    xaxis=dict(title='Building'),
    yaxis=dict(title='Metric-Tons of CO2 Emitted', type='log'),
    barmode='stack',
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Show plot
fig.show()
